In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
import os
root_path = 'gdrive/My Drive/EACL/'
os.chdir(root_path)

Mounted at /content/gdrive


In [2]:
!pip install sentencepiece
!pip install transformers
!pip install demoji

     |████████████████████████████████| 1.2MB 8.4MB/s 
     |████████████████████████████████| 1.8MB 8.5MB/s 
     |████████████████████████████████| 2.9MB 46.3MB/s 
     |████████████████████████████████| 890kB 57.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ff70f238c7b61de5e1285705fcf3f25ca8e52706e3cb664a28537b1b72d049be
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!pip install git+git://github.com/irshadbhat/indic-trans.git

  Cloning git://github.com/irshadbhat/indic-trans.git to /tmp/pip-req-build-o6iqen7f
  Running command git clone -q git://github.com/irshadbhat/indic-trans.git /tmp/pip-req-build-o6iqen7f
  Using cached https://files.pythonhosted.org/packages/fb/48/69046506f6ac61c1eaa9a0d42d22d54673b69e176d30ca98e3f61513e980/pbr-5.5.1-py2.py3-none-any.whl
  Created wheel for indictrans: filename=indictrans-1.2.3-cp36-cp36m-linux_x86_64.whl size=337563815 sha256=5051ba4a7486019827b63c3e4e29bf1939b63d922691eafb9787af6b71e9975f
  Stored in directory: /tmp/pip-ephem-wheel-cache-qo7g7qb2/wheels/01/a9/02/ca41463296c4ea023505525a9e355c68f3a71225c81495e6d7
Successfully built indictrans


In [4]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import copy
from transformers import BertModel, RobertaModel, BertTokenizer, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, random_split, DataLoader, IterableDataset, ConcatDataset
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score 
from tqdm import tqdm
import demoji 
from indictrans import Transliterator
demoji.download_codes() 
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams.update({'font.size': 8})
RANDOM_SEED = 42
path = Path('./')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

... OK (Got response in 0.16 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [5]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.7.0+cu101')

In [6]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
random_seed(RANDOM_SEED, True)

In [7]:
def handle_all_caps(t: str) -> str:
    tokens = t.split()
    tokens = replace_all_caps(tokens)
    return ' '.join(tokens)

def handle_upper_case_first_letter(t: str) -> str:
    tokens = t.split()
    tokens = deal_caps(tokens)
    return ' '.join(tokens)

def lower_case_everything(t: str) -> str:
    return t.lower().replace('@user', '').replace('#tag ', '').replace('rt ', '').strip()

In [8]:
class CodeMixedKanTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str(path/"./Kan/kannada_lm.model"))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

sp = spm.SentencePieceProcessor()
sp.Load(str(path/"./Kan/kannada_lm.model"))
itos = [sp.IdToPiece(int(i)) for i in range(25000)]

In [9]:
itos[:20]

['<unk>',
 '<s>',
 '</s>',
 '.',
 ',',
 '▁',
 '▁ಮತ್ತು',
 'ದ',
 '▁ಈ',
 'ಗಳು',
 'ಯ',
 'ಗಳ',
 'ಗಳನ್ನು',
 'ರ',
 '▁ಒಂದು',
 'ವನ್ನು',
 '-',
 'ವು',
 'ನ',
 'ರು']

In [10]:
class Dataset():
    def __init__(self, train_data, val_data, test_data):
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        # self.batch_size = batch_size

        self.label_dict = {}
        
        self.count_dic = {}

        self.trn = Transliterator(source='eng', target='kan', build_lookup=True)

        self.train_df = pd.DataFrame(self.process_data(self.train_data))
        self.val_df = pd.DataFrame(self.process_data(self.val_data))
        self.test_df = pd.DataFrame(self.process_data(self.test_data, test = True))

    def is_english(self, s: str) -> bool:
        if len(re.findall(u'[\u0900-\u097F]', s)) <= 1000:
            return True
        return False

    def process_data(self, data, test = False):
        sentences, labels = [], []
        print(len(data))
        for line in data:
            sentence = line.strip().split('\t')
            if not test:
                label = sentence.pop()
                if label not in self.label_dict:
                    self.label_dict[label] = len(self.label_dict)
                labels.append(self.label_dict[label])
                self.count_dic[label] = self.count_dic.get(label, 0) + 1
            sentence = ' '.join(sentence)
            emoji_dict = demoji.findall(sentence)

            # if not self.is_english(sentence):
            #     print(sentence)
            #     sentence = self.trn.transform(sentence)
            #     print(sentence)

            sentence = self.trn.transform(sentence)
            # print(sentence)

            if len(emoji_dict): 
                for emoji, text in emoji_dict.items():
                    sentence = sentence.replace(emoji, ' '+text+' ')
                    sentence = ' '.join(sentence.split())
            
            sentences.append(sentence)
            
            # labels.append(label)
            
        if test: return {'input': sentences}
        return {'input': sentences, 'label': (labels)}

In [11]:
with open('Dataset/kannada_offensive_train.csv', 'r') as f:
    train_data = f.readlines()
with open('Dataset/kannada_offensive_dev.csv', 'r') as f:
    val_data = f.readlines()
with open('Dataset/kannada_offensive_test.csv', 'r') as f:
    test_data = f.readlines()
data = Dataset(train_data, val_data, test_data)
data.label_dict

6217
777
778


{'Not_offensive': 0,
 'Offensive_Targeted_Insult_Group': 2,
 'Offensive_Targeted_Insult_Individual': 4,
 'Offensive_Targeted_Insult_Other': 3,
 'Offensive_Untargetede': 5,
 'not-Kannada': 1}

In [12]:
data.test_df.head()

,input
0,ಜೈ ಮೋರಿ ಜೈ ರೋಗಿ ಇಸ್ಲಾಂ ಸಾಯಿಸಿ ಇದೆ ಇವರ ಗುರಿ
1,ಅನ್ನಾ ನಿಮ ಎ ವೇದಿಯೋ ನೋದೀನುಂ ಮಾಥೇ ಚೈನಾ ಆಪ್ಪ ಉಸ್ ...
2,ಸುಪರ್ ಅನ್ನಾ ಯೇನುಂ ಥಾಪ್ಪಿಲಾ ಯೇಲಾ ಸಾರಿ ಅಗೀ ಹೆಲಿದ...
3,ಸುಪರ್ ಸಿರ ಸುಪರ್ ಸುಪರ್ ಸುಪರ್ ಸುಪರ್
4,ಆಗ್ತ-ಕೇಲು (ಕನ್ನಡ ರಪ 2019)


In [13]:
from collections import Counter
Counter(data.train_df['label'])

Counter({0: 3544, 1: 1522, 2: 329, 3: 123, 4: 487, 5: 212})

In [14]:
label_cols = ['label']
text_cols = ['input']

In [15]:
taen_vocab = Vocab(itos)
tokenizer = Tokenizer(lang='kn', tok_func=CodeMixedKanTokenizer)
tokenizer.pre_rules.append(lower_case_everything)
tokenizer.pre_rules.append(handle_all_caps)
tokenizer.pre_rules.append(handle_upper_case_first_letter)

In [16]:
tokenizer.special_cases, tokenizer.pre_rules, tokenizer.post_rules

(['xxunk',
  'xxpad',
  'xxbos',
  'xxeos',
  'xxfld',
  'xxmaj',
  'xxup',
  'xxrep',
  'xxwrep'],
 [<function fastai.text.transform.fix_html>,
  <function fastai.text.transform.replace_rep>,
  <function fastai.text.transform.replace_wrep>,
  <function fastai.text.transform.spec_add_spaces>,
  <function fastai.text.transform.rm_useless_spaces>,
  <function __main__.lower_case_everything>,
  <function __main__.handle_all_caps>,
  <function __main__.handle_upper_case_first_letter>],
 [<function fastai.text.transform.replace_all_caps>,
  <function fastai.text.transform.deal_caps>])

In [17]:
tokens = tokenizer.process_all(['Tell me about TOUR self, mujhe jaanna hai'])
''.join(tokens[0])

'▁tell▁me▁about▁tour▁self,▁mujhe▁jaanna▁hai'

In [18]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=data.train_df, valid_df=data.val_df, test_df=data.test_df, tokenizer=tokenizer, vocab=taen_vocab, label_cols=label_cols, text_cols=text_cols)

/usr/local/lib/python3.6/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [19]:
data_lm.show_batch()

/usr/local/lib/python3.6/dist-packages/fastai/text/data.py:339: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  idx_min = (t != self.pad_idx).nonzero().min()


idx,text
0,ನ್ ▁ಪ್ ಹೈ ಮ ▁ಅಲ್ಲ ೀ ▁ಬಿಟ್ಟ ಿದ ಿ ರಾ ▁x x re p ▁4 ▁ . ▁ಯ ೆಲ್ಲಾ ▁ಮ ನೇ ಲೆ ▁ನೋಡ ್ ತಾ ರೇ ▁x x re p ▁4 ▁ . ▁x x b os ▁ಗುರು ▁ಸೂ ರೇ ▁ನ್ಯೂಸ್ ▁ನ ನು ▁ ಟಿಕ ▁ಟೋ ಕ ▁ದಾನ ▁ಲೋಡ್ ▁ಮ ದೇ ದಾ ▁ಯ ವಾಗ ಾ ▁ನಾಮ ಾ ▁ ನ್ ದ ುದು ▁ತಿಂದು ▁ನಾಮ ಾ ▁ದ ಶ್ ದಾ ▁ ಮಾಲಾ ▁ಉದ್
1,ೀ ▁ಸಾ ಈ ▁ಕುಮಾರ ▁ ದೀ ಅ ಲಾ ಜ್ ▁x x b os ▁ಹ್ಯಾಂಡ್ ಸ್ ▁ಉಪ ▁ಟು ▁ವಿಜಯಾ ▁ಪ್ರಕಾಶ ▁ಸಿ ರ ▁x x b os ▁ಏನ ▁ಕ ಮೆಂಟ್ ▁ಗುರು ▁ಎಲ್ ▁ಇ ರ್ ತೀ ರೋ ▁ನೀ ವೆಲ್ಲಾ ▁ಸೂಪರ್ ▁x x b os ▁ಹ್ಯಾಂಡ್ ಸ್ ▁ಉಪ ▁- ▁30 8 ಕ ▁ಲೈ ಕ್ಸ್ ▁ ನ್ ಬೇ ಲೆ ಗೆ ಡ್ ದು ▁- ▁30 8 ಕ ▁ಲೈ ಕ್ಸ್ ▁x x b os ▁ಬೆಸ್ಟ್ ▁ಎಪಿ ಸೋ
2,▁f a ce ▁co w b o y ▁ hat ▁f a ce ▁x x b os ▁ಎ ▁ಮತ ು ▁x x b os ▁ದ ಏ ▁ಗೇ ▁600 ಕ <unk> ▁ವಿ ಊ ಸ್ ▁ನೇ ನ ▁ಹಾ ವ್ ಲಿ ▁ಗುರು ▁x x b os ▁ <unk> ಟ್ರೋ ಲ್ ▁ಮಾನ್ ಸ್ಟರ್ <unk> ಕನ್ನಡ ▁ನಿ ಮ ▁ಅ ಮಾನ್ ▁ಕಲ್ ಸು ▁ ಥೊ ರಿಸ್ ಥೀ ನೀ ▁x x b os ▁ಸಿ ಗ್ ನ್ಯಾ ತುರ್
3,ರಾಗ ೀ ದ ರೆ ▁x x b os ▁ಸುಪರ್ ▁ಮೂ ವಿ ▁ಐ ▁ಲೈ ಕ್ ▁ಇ ತ ▁x x b os ▁ ಯ್ಸ್ ▁ ಯ್ಸ್ ▁ ಯ್ಸ್ ▁ lo ve - y ou ▁g est ure ▁x x b os ▁ಸು ಪೆರ್ ಬ್ ▁ಸೋ ಂಗ್ ▁... ▁ಲೀ ಕೇತ ▁ಇ ತ ▁x x b os ▁ <unk> ಗ ▁ದ ▁ಯ ತೀ ಶ ▁ಗುಡ್ ▁ಜೋ ಬ್ ▁ಬ್ರೊ ▁x x b os ▁ಯಾ ▁ಬ್ರೊ
4,ಪ್ಪ ೋರ್ ಟಿಂಗ್ ▁ಕನ್ನಡ ▁ಮೂ ವಿಸ್ . ▁ . ▁x x b os ▁ಬಿಂದು ▁ಗೋ ವ ಡಾ ▁ಲೋ ಫೆ ರ್ ▁ಅವ ಲು ▁x x b os ▁ಗುರು ▁ಅಶ್ವ ತ್ತಾ ಮಾ ▁ನ ▁ಜೋ ತ್ ▁ಅವರ ▁ಅಪ್ಪ ಾ ▁ಕು ಡಾ ▁ಬಂಡಿ ದ್ ರು ▁ನ ಮ ▁ಉರ್ ಜ್ ▁ಗೊ ಟಿ ಲ್ ವಾ ▁ನಿ ಂಗ ೇ ▁ಕೃಷ್ಣ ▁ಕು ಡಾ ▁ಸಿಕ್ಕ ಿದ್ದ ಾ ▁x x b os ▁ಪ್ರ ಜ್ಜ ೂ ▁ಬಾಸ್ ▁ಇಸ್ ▁ಬ ಕ ▁x


In [20]:

awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False,  model_dir = './Kan/models/')
# learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, pretrained=False,

In [ ]:
learn.load('third_kn_lm',with_opt=True, device=device)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (6217 items)
x: LMTextList
▁x x b os ▁ ಟಿಕ ▁ಟೋ ಕ ▁ಅಲ್ಲ ೀ ▁ಜಗ ಲಾ ▁ಮಡ ಇ ದ್ರ ಲ್ಲಾ ▁ಆದ ್ರಾ ▁ಬಗ್ಗ ೇ ನು ಂ ▁ಓ ಂದು ▁ವಿಡಿಯೋ ▁ಮ ದಿ ▁ಅನ್ನಾ ▁ಸುಪರ್ ▁ಅ ಗೀ ▁ಉ ಗೀ ದೀ ದ್ದ ೀ ರಾ,▁x x b os ▁ಅನ್ಯ ೋ ನೆ ▁ಫ್ರ ್ಮ ▁ಕೇರಳ ಾ ▁ಹೀ ರಾ,▁x x b os ▁ಮೂ ವಿ ▁ರೇ ರೆ ಲೀಸ್ ▁ಮ ದಿ ▁ಪ್ ಲ್ಸ್,▁x x b os ▁ಅ ಮಾ ಝೋ ನ್ ▁ಪ್ ಹೈ ಮ ▁ಅಲ್ಲ ೀ ▁ಬಿಟ್ಟ ಿದ ಿ ರಾ ▁x x re p ▁4 ▁ . ▁ಯ ೆಲ್ಲಾ ▁ಮ ನೇ ಲೆ ▁ನೋಡ ್ ತಾ ರೇ ▁x x re p ▁4 ▁ .,▁x x b os ▁ಗುರು ▁ಸೂ ರೇ ▁ನ್ಯೂಸ್ ▁ನ ನು ▁ ಟಿಕ ▁ಟೋ ಕ ▁ದಾನ ▁ಲೋಡ್ ▁ಮ ದೇ ದಾ ▁ಯ ವಾಗ ಾ ▁ನಾಮ ಾ ▁ ನ್ ದ ುದು ▁ತಿಂದು ▁ನಾಮ ಾ ▁ದ ಶ್ ದಾ ▁ ಮಾಲಾ ▁ಉದ್ ಧಾ ▁ಮ ದಾ ಕಾ ▁ಬ ಂಡ್ ರೋ ▁ಅವ ಗಾ ▁ಡೆ ಲೆ ಟೆಡ್
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (777 items)
x: LMTextList
▁x x b os ▁100 ▁ಡೇ ಸ್ ▁ಪಕ್ಕ ಾ,▁x x b os ▁ಸುಪರ್ ▁ಸಿ ರ,▁x x b os ▁ಹ್ಯಾಂಡ್ ಸ್ ▁ಉಪ,▁x x b os ▁ಸಾರಿ ▁ನೀವು ▁ವಿಡಿಯೋ ▁ನ ▁ರೋಸ್ ತ ▁ಮ ದಿ ▁ಅ ದ್ ರೇ ▁ಮ ದ್ವಾ ಗಾ ▁ಪು ಬ್ ಗ ▁ಅ ತಾ ವಾ ▁ಫ್ರೀ ▁ಫೈರ್ ▁ಗೇಮ್ಸ್ ▁ಹ ಕೋ ಂಡೋ ▁ ಮದ್ ಬೇ ದೀ ▁ಆ ▁ಗೇಮ್ಸ್ ▁ಕೂಡಾ ▁ಚೀನ ್,▁x x b os ▁ಕ್ರಿ ಶಾ ನಾ ▁ಶಾಪ ಾ ▁ಟಾ ಟ್ಟೆ ಲಿ ಬೇಕು
y: LMLabelList
,,,,
Path: .;

Test: LabelList (778 items)

In [ ]:
learn.freeze()
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.441038,4.887317,0.291473,00:07


In [ ]:
learn.save('fit_head', with_opt=True)

In [ ]:
learn.load('fit_head', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (6217 items)
x: LMTextList
▁x x b os ▁ ಟಿಕ ▁ಟೋ ಕ ▁ಅಲ್ಲ ೀ ▁ಜಗ ಲಾ ▁ಮಡ ಇ ದ್ರ ಲ್ಲಾ ▁ಆದ ್ರಾ ▁ಬಗ್ಗ ೇ ನು ಂ ▁ಓ ಂದು ▁ವಿಡಿಯೋ ▁ಮ ದಿ ▁ಅನ್ನಾ ▁ಸುಪರ್ ▁ಅ ಗೀ ▁ಉ ಗೀ ದೀ ದ್ದ ೀ ರಾ,▁x x b os ▁ಅನ್ಯ ೋ ನೆ ▁ಫ್ರ ್ಮ ▁ಕೇರಳ ಾ ▁ಹೀ ರಾ,▁x x b os ▁ಮೂ ವಿ ▁ರೇ ರೆ ಲೀಸ್ ▁ಮ ದಿ ▁ಪ್ ಲ್ಸ್,▁x x b os ▁ಅ ಮಾ ಝೋ ನ್ ▁ಪ್ ಹೈ ಮ ▁ಅಲ್ಲ ೀ ▁ಬಿಟ್ಟ ಿದ ಿ ರಾ ▁x x re p ▁4 ▁ . ▁ಯ ೆಲ್ಲಾ ▁ಮ ನೇ ಲೆ ▁ನೋಡ ್ ತಾ ರೇ ▁x x re p ▁4 ▁ .,▁x x b os ▁ಗುರು ▁ಸೂ ರೇ ▁ನ್ಯೂಸ್ ▁ನ ನು ▁ ಟಿಕ ▁ಟೋ ಕ ▁ದಾನ ▁ಲೋಡ್ ▁ಮ ದೇ ದಾ ▁ಯ ವಾಗ ಾ ▁ನಾಮ ಾ ▁ ನ್ ದ ುದು ▁ತಿಂದು ▁ನಾಮ ಾ ▁ದ ಶ್ ದಾ ▁ ಮಾಲಾ ▁ಉದ್ ಧಾ ▁ಮ ದಾ ಕಾ ▁ಬ ಂಡ್ ರೋ ▁ಅವ ಗಾ ▁ಡೆ ಲೆ ಟೆಡ್
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (777 items)
x: LMTextList
▁x x b os ▁100 ▁ಡೇ ಸ್ ▁ಪಕ್ಕ ಾ,▁x x b os ▁ಸುಪರ್ ▁ಸಿ ರ,▁x x b os ▁ಹ್ಯಾಂಡ್ ಸ್ ▁ಉಪ,▁x x b os ▁ಸಾರಿ ▁ನೀವು ▁ವಿಡಿಯೋ ▁ನ ▁ರೋಸ್ ತ ▁ಮ ದಿ ▁ಅ ದ್ ರೇ ▁ಮ ದ್ವಾ ಗಾ ▁ಪು ಬ್ ಗ ▁ಅ ತಾ ವಾ ▁ಫ್ರೀ ▁ಫೈರ್ ▁ಗೇಮ್ಸ್ ▁ಹ ಕೋ ಂಡೋ ▁ ಮದ್ ಬೇ ದೀ ▁ಆ ▁ಗೇಮ್ಸ್ ▁ಕೂಡಾ ▁ಚೀನ ್,▁x x b os ▁ಕ್ರಿ ಶಾ ನಾ ▁ಶಾಪ ಾ ▁ಟಾ ಟ್ಟೆ ಲಿ ಬೇಕು
y: LMLabelList
,,,,
Path: .;

Test: LabelList (778 items)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.660072,4.513894,0.322277,00:09
1,4.376020,4.157182,0.367545,00:09
2,4.117792,4.005173,0.386741,00:09
3,3.914269,3.952537,0.395045,00:09
4,3.790273,3.945087,0.396205,00:10


In [ ]:
learn.save('fine_tuned', with_opt=True)

In [ ]:
learn.load('fine_tuned', with_opt=True);
learn.predict('en payar',n_words=10)

'en payar ▁f a ce ▁with ▁he art s ▁f a ce'

In [ ]:
learn.save_encoder('fine_tuned_enc')

In [ ]:
def change_1h_targ(targ):
    n = len(data.label_dict)
    ohe = [0]*n
    ohe[targ] = 1
    return ohe

In [ ]:
data.train_df['ohe'] = data.train_df['label'].apply(change_1h_targ)
data.val_df['ohe'] = data.val_df['label'].apply(change_1h_targ)
label_cols = ['label']
text_cols = ['input']

In [ ]:
data.train_df

,input,label,ohe
0,ಟಿಕ ಟೋಕ ಅಲ್ಲೀ ಜಗಲಾ ಮಡಇದ್ರಲ್ಲಾ ಆದ್ರಾ ಬಗ್ಗೇನುಂ ಓ...,0,"[1, 0, 0, 0, 0, 0]"
1,ಅನ್ಯೋನೆ ಫ್ರ್ಮ ಕೇರಳಾ ಹೀರಾ,1,"[0, 1, 0, 0, 0, 0]"
2,ಮೂವಿ ರೇರೆಲೀಸ್ ಮದಿ ಪ್ಲ್ಸ್,0,"[1, 0, 0, 0, 0, 0]"
3,ಅಮಾಝೋನ್ ಪ್ಹೈಮ ಅಲ್ಲೀ ಬಿಟ್ಟಿದಿರಾ....ಯೆಲ್ಲಾ ಮನೇಲೆ...,0,"[1, 0, 0, 0, 0, 0]"
4,ಗುರು ಸೂರೇ ನ್ಯೂಸ್ ನನು ಟಿಕ ಟೋಕ ದಾನ ಲೋಡ್ ಮದೇದಾ ಯವ...,0,"[1, 0, 0, 0, 0, 0]"
...,...,...,...
6212,@ಗಜೇಂದ್ರ ಹವ್ ನೀನ ಏನ ಮಾದಿರೋಡುಂ ಮೇಲೇ,4,"[0, 0, 0, 0, 1, 0]"
6213,ಸಿರ ನೀವ್ ಅದ್ಥೀಡ್ಡಾ ಗಮ್ ಯಾವದು,0,"[1, 0, 0, 0, 0, 0]"
6214,ಡೂಬ್ಬ ಫೂಲ ಮೂವಿ ಇನ್ ತೇಲುಗು,0,"[1, 0, 0, 0, 0, 0]"
6215,ಮೋಸ್ಟ್ಲಿ ನಿಮ್ಮಾಂ ಮಟೆ ಕೇಲಿ ಟಿಕ್ ಟೋಕ ಬನ ಅಗೀರ್ಬೇಕ...,0,"[1, 0, 0, 0, 0, 0]"


In [21]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=data.train_df, valid_df=data.val_df, test_df = data.test_df, tokenizer=tokenizer, vocab=taen_vocab, bs=16, label_cols = label_cols, text_cols=text_cols)
data_clas.show_batch()

/usr/local/lib/python3.6/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


text,target
▁x x b os ▁ <unk> ಪ ೃ ಥ್ ವೀ ▁ನಾಯಕ ▁ನಿಮ್ಮ ▁ಅ ವ್ವ ▁ಊರ ್ ▁ಹಾ ದ ರಗಿ ತ್ತಿ ▁ಅಂತ ೇ ▁r oll ing ▁on ▁the ▁f lo or ▁ la ug h ing ▁r oll ing ▁on ▁the ▁f lo or ▁ la ug h ing ▁r oll ing ▁on ▁the ▁f lo or ▁ la ug h ing ▁r oll ing ▁on ▁the ▁f lo or ▁,4
▁x x b os ▁ಲಾಕ್ ▁ಡೌನ್ ▁ಸಮಯದಲ್ಲಿ ▁ಕಾರ್ಮಿಕರು ▁ವಲಸಿಗರ ು ▁ ರೆಸ್ ತೆ ▁ಮೇಲೆ ▁ನಡೆದುಕೊಂಡು ▁ಹೋಗುತ್ತಿರುವ ▁ ವಿಡೀ ಯೋ ಗಳನ್ನ ▁ತೀ ಕ್ತ ಕ ▁ನಲ್ಲಿ ▁ಜನರಿಗೆ ▁ತೋರಿಸ ಲಾಯ ಿತ್ತು ▁x x re p ▁4 ▁ . ▁ಈ ▁ತೀ ಕ್ತ ಕ ▁ಇಲ್ಲ ▁ಅಂದ ಿದ್ದರೆ ▁ಈ ▁ದಳ್ಳಾಳಿ ▁ಮಿ ಡಿಯಾ ▁ಬು ಲೆಟ್ ▁ಟ್ರೈ ನ್ ▁ಅಲ್ಲಿ ▁ಹೋಗುವ ▁ಆ ಗೆ ▁ಜನರಿಗೆ ▁ತೋರಿಸ ುತ್ತಿದ್ದರು . . . ▁ಜನರಿಂದ ▁ಜನರಿಗೆ ▁ಮಾಹಿತಿ ▁ತಿಳಿಯ ುತ್ತಿತ್ತು . . . ▁ಇದರಲ್ಲಿ ▁ಸಾ,0
▁x x b os ▁ನ ಮಸ್ ತೆ ▁ಮಸ್ ತ್ ▁ಮಗ ಾ . ಕಾಮ್ ನ್ ನಿಮ್ಮ ▁ಚಾನಲ್ ▁ಅಭಿಮಾನಿ ಗಳಲ್ಲೊಬ್ಬ ▁ನಾನು . . . ನ ನಿಮ್ಮ ▁ವಿಡಿಯೋ ಗಳಿಗಾಗಿ ▁ಕಾಯ ್ತಿ ರ್ ತೇ ನೆ . . . ನ್ ಈ ▁ವಿಡಿಯೋ ▁ನೋಡಿ ▁ಯಾ ಕೋ ▁ಒಂದಷ್ಟು ▁ಚರ್ಚೆ ▁ಮಾಡ ಬಹುದೇ ನೋ ▁ಅನ್ನಿ ಸ್ತು . . . ▁ನನಗ ನ ಿಸಿದ್ದ ನ್ನ ▁ಹೇಳ ್ತಿ ದೇ ನೆ . . . ▁ನ ಸರ್ಕಾರ ಗಳ ▁ನಿರ್ಲಕ್ಷ್ಯ ದ ▁ಬಗ್ಗೆ ▁ನೀ ವಂದ ದ್ದು,0
▁x x b os ▁ನಮ್ಮ ▁ದೇಶದಲ್ಲಿ ▁ಸ್ವಲ್ಪ ▁ಜನ ▁ದೇಶದ ▁ಬಗ್ಗೆ ▁ಯೋಚನೆ ▁ಮಾಡ ೋ ದು ▁ಬಿಟ್ಟ ▁ . ▁ಬಕೆಟ್ ▁ಹಿಡಿಯ ೋ ▁ಕೆಲಸ ▁ಚ ನ್ನಾಗಿ ▁ಮಾಡ ್ ತಾರೆ ▁ಅಣ್ಣಾ ▁ಅದಕ್ಕೆ ▁ಹೀಗೆ ಲ್ಲ ▁ಮಾಡ ್ ತಾರೆ ▁ಅವರು ▁x x re p ▁4 ▁ . ▁ ನ್ ದೇಶ ಕಾ ಗೀ ▁ಜೀವ ▁ಕೋ ತ್ತಾ ▁ಆ ▁ಯೋಧರ ೀಗ ೀನ ತಾ ▁ಎ ▁ತ ಲೇ ಂತ ▁ಅಂಕ ೋ ಂಡ್ ▁ ಯೆಲ್ ಲಾರ್ ಗು ▁ಇ ರ್ ರಿ ಟಾ ಟ್,5
▁x x b os ▁ಇ ವ್ ರ ಂತು ▁ಉ ನ ▁ಇನ್ ಸ್ಟಾಲ್ ▁ಮಡ ಲ್ಲಾ ▁ಸು ಲೇ ಮಾ ಕ್ ಲೂ . ▁ಅವ ್ರ ನಾ ▁ಫೋ ಲೌ ಇ ಂಗ್ ▁ಮ ಡ್ ಟಿ ರೊ ▁ನ ಯ ೀಗ ಲಿಗೆ ▁ಬದ್ದ ೀ ▁ಬಾರ್ ಬೇಕು ▁ಮೋಡ ಲು ಂ . ▁ನ ಮ ▁ಇಂಡಿಯಾ ▁ ದಲ್ಲಿ ರೋ ▁ಇಂ ತಾ ▁ನ ಯಿ ಗಾ ಲಾ ▁ಕ ಥ ೇ ▁ನೇ ▁ಇ ಸ್ತು ▁ದೇಶ ಾದ ▁ಬಾಗ್ ಗೇ ▁ಯೆ ನಾದ ರು ▁ಗೊ ಟಿ,5


In [22]:
data_clas.sanity_check()

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [23]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [24]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config,model_dir = './Kan/models')
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (6217 items)
x: TextList
▁x x b os ▁ ಟಿಕ ▁ಟೋ ಕ ▁ಅಲ್ಲ ೀ ▁ಜಗ ಲಾ ▁ಮಡ ಇ ದ್ರ ಲ್ಲಾ ▁ಆದ ್ರಾ ▁ಬಗ್ಗ ೇ ನು ಂ ▁ಓ ಂದು ▁ವಿಡಿಯೋ ▁ಮ ದಿ ▁ಅನ್ನಾ ▁ಸುಪರ್ ▁ಅ ಗೀ ▁ಉ ಗೀ ದೀ ದ್ದ ೀ ರಾ,▁x x b os ▁ಅನ್ಯ ೋ ನೆ ▁ಫ್ರ ್ಮ ▁ಕೇರಳ ಾ ▁ಹೀ ರಾ,▁x x b os ▁ಮೂ ವಿ ▁ರೇ ರೆ ಲೀಸ್ ▁ಮ ದಿ ▁ಪ್ ಲ್ಸ್,▁x x b os ▁ಅ ಮಾ ಝೋ ನ್ ▁ಪ್ ಹೈ ಮ ▁ಅಲ್ಲ ೀ ▁ಬಿಟ್ಟ ಿದ ಿ ರಾ ▁x x re p ▁4 ▁ . ▁ಯ ೆಲ್ಲಾ ▁ಮ ನೇ ಲೆ ▁ನೋಡ ್ ತಾ ರೇ ▁x x re p ▁4 ▁ .,▁x x b os ▁ಗುರು ▁ಸೂ ರೇ ▁ನ್ಯೂಸ್ ▁ನ ನು ▁ ಟಿಕ ▁ಟೋ ಕ ▁ದಾನ ▁ಲೋಡ್ ▁ಮ ದೇ ದಾ ▁ಯ ವಾಗ ಾ ▁ನಾಮ ಾ ▁ ನ್ ದ ುದು ▁ತಿಂದು ▁ನಾಮ ಾ ▁ದ ಶ್ ದಾ ▁ ಮಾಲಾ ▁ಉದ್ ಧಾ ▁ಮ ದಾ ಕಾ ▁ಬ ಂಡ್ ರೋ ▁ಅವ ಗಾ ▁ಡೆ ಲೆ ಟೆಡ್
y: CategoryList
0,1,0,0,0
Path: .;

Valid: LabelList (777 items)
x: TextList
▁x x b os ▁100 ▁ಡೇ ಸ್ ▁ಪಕ್ಕ ಾ,▁x x b os ▁ಸುಪರ್ ▁ಸಿ ರ,▁x x b os ▁ಹ್ಯಾಂಡ್ ಸ್ ▁ಉಪ,▁x x b os ▁ಸಾರಿ ▁ನೀವು ▁ವಿಡಿಯೋ ▁ನ ▁ರೋಸ್ ತ ▁ಮ ದಿ ▁ಅ ದ್ ರೇ ▁ಮ ದ್ವಾ ಗಾ ▁ಪು ಬ್ ಗ ▁ಅ ತಾ ವಾ ▁ಫ್ರೀ ▁ಫೈರ್ ▁ಗೇಮ್ಸ್ ▁ಹ ಕೋ ಂಡೋ ▁ ಮದ್ ಬೇ ದೀ ▁ಆ ▁ಗೇಮ್ಸ್ ▁ಕೂಡಾ ▁ಚೀನ ್,▁x x b os ▁ಕ್ರಿ ಶಾ ನಾ ▁ಶಾಪ ಾ ▁ಟಾ ಟ್ಟೆ ಲಿ ಬೇಕು
y: CategoryList
0,1,1,0,0
Path: .;

Test: LabelList (778 i

In [ ]:
learn.freeze()
learn.loss_func.func

CrossEntropyLoss()

In [ ]:
f1 = FBeta(beta = 1,average = 'weighted')

In [ ]:
# learn.metrics = [mcc, accuracy]
learn.metrics = [accuracy, f1]

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,f_beta,time
0,0.983704,0.893651,0.685972,0.640928,00:08


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn.save('first-full')

In [ ]:
learn.load('first-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (6217 items)
x: TextList
▁x x b os ▁ ಟಿಕ ▁ಟೋ ಕ ▁ಅಲ್ಲ ೀ ▁ಜಗ ಲಾ ▁ಮಡ ಇ ದ್ರ ಲ್ಲಾ ▁ಆದ ್ರಾ ▁ಬಗ್ಗ ೇ ನು ಂ ▁ಓ ಂದು ▁ವಿಡಿಯೋ ▁ಮ ದಿ ▁ಅನ್ನಾ ▁ಸುಪರ್ ▁ಅ ಗೀ ▁ಉ ಗೀ ದೀ ದ್ದ ೀ ರಾ,▁x x b os ▁ಅನ್ಯ ೋ ನೆ ▁ಫ್ರ ್ಮ ▁ಕೇರಳ ಾ ▁ಹೀ ರಾ,▁x x b os ▁ಮೂ ವಿ ▁ರೇ ರೆ ಲೀಸ್ ▁ಮ ದಿ ▁ಪ್ ಲ್ಸ್,▁x x b os ▁ಅ ಮಾ ಝೋ ನ್ ▁ಪ್ ಹೈ ಮ ▁ಅಲ್ಲ ೀ ▁ಬಿಟ್ಟ ಿದ ಿ ರಾ ▁x x re p ▁4 ▁ . ▁ಯ ೆಲ್ಲಾ ▁ಮ ನೇ ಲೆ ▁ನೋಡ ್ ತಾ ರೇ ▁x x re p ▁4 ▁ .,▁x x b os ▁ಗುರು ▁ಸೂ ರೇ ▁ನ್ಯೂಸ್ ▁ನ ನು ▁ ಟಿಕ ▁ಟೋ ಕ ▁ದಾನ ▁ಲೋಡ್ ▁ಮ ದೇ ದಾ ▁ಯ ವಾಗ ಾ ▁ನಾಮ ಾ ▁ ನ್ ದ ುದು ▁ತಿಂದು ▁ನಾಮ ಾ ▁ದ ಶ್ ದಾ ▁ ಮಾಲಾ ▁ಉದ್ ಧಾ ▁ಮ ದಾ ಕಾ ▁ಬ ಂಡ್ ರೋ ▁ಅವ ಗಾ ▁ಡೆ ಲೆ ಟೆಡ್
y: CategoryList
0,1,0,0,0
Path: .;

Valid: LabelList (777 items)
x: TextList
▁x x b os ▁100 ▁ಡೇ ಸ್ ▁ಪಕ್ಕ ಾ,▁x x b os ▁ಸುಪರ್ ▁ಸಿ ರ,▁x x b os ▁ಹ್ಯಾಂಡ್ ಸ್ ▁ಉಪ,▁x x b os ▁ಸಾರಿ ▁ನೀವು ▁ವಿಡಿಯೋ ▁ನ ▁ರೋಸ್ ತ ▁ಮ ದಿ ▁ಅ ದ್ ರೇ ▁ಮ ದ್ವಾ ಗಾ ▁ಪು ಬ್ ಗ ▁ಅ ತಾ ವಾ ▁ಫ್ರೀ ▁ಫೈರ್ ▁ಗೇಮ್ಸ್ ▁ಹ ಕೋ ಂಡೋ ▁ ಮದ್ ಬೇ ದೀ ▁ಆ ▁ಗೇಮ್ಸ್ ▁ಕೂಡಾ ▁ಚೀನ ್,▁x x b os ▁ಕ್ರಿ ಶಾ ನಾ ▁ಶಾಪ ಾ ▁ಟಾ ಟ್ಟೆ ಲಿ ಬೇಕು
y: CategoryList
0,1,1,0,0
Path: .;

Test: LabelList (778 i

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,f_beta,time
0,0.943596,0.895889,0.689833,0.654704,00:09


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn.save('second-full')

In [ ]:
learn.load('second-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (6217 items)
x: TextList
▁x x b os ▁ ಟಿಕ ▁ಟೋ ಕ ▁ಅಲ್ಲ ೀ ▁ಜಗ ಲಾ ▁ಮಡ ಇ ದ್ರ ಲ್ಲಾ ▁ಆದ ್ರಾ ▁ಬಗ್ಗ ೇ ನು ಂ ▁ಓ ಂದು ▁ವಿಡಿಯೋ ▁ಮ ದಿ ▁ಅನ್ನಾ ▁ಸುಪರ್ ▁ಅ ಗೀ ▁ಉ ಗೀ ದೀ ದ್ದ ೀ ರಾ,▁x x b os ▁ಅನ್ಯ ೋ ನೆ ▁ಫ್ರ ್ಮ ▁ಕೇರಳ ಾ ▁ಹೀ ರಾ,▁x x b os ▁ಮೂ ವಿ ▁ರೇ ರೆ ಲೀಸ್ ▁ಮ ದಿ ▁ಪ್ ಲ್ಸ್,▁x x b os ▁ಅ ಮಾ ಝೋ ನ್ ▁ಪ್ ಹೈ ಮ ▁ಅಲ್ಲ ೀ ▁ಬಿಟ್ಟ ಿದ ಿ ರಾ ▁x x re p ▁4 ▁ . ▁ಯ ೆಲ್ಲಾ ▁ಮ ನೇ ಲೆ ▁ನೋಡ ್ ತಾ ರೇ ▁x x re p ▁4 ▁ .,▁x x b os ▁ಗುರು ▁ಸೂ ರೇ ▁ನ್ಯೂಸ್ ▁ನ ನು ▁ ಟಿಕ ▁ಟೋ ಕ ▁ದಾನ ▁ಲೋಡ್ ▁ಮ ದೇ ದಾ ▁ಯ ವಾಗ ಾ ▁ನಾಮ ಾ ▁ ನ್ ದ ುದು ▁ತಿಂದು ▁ನಾಮ ಾ ▁ದ ಶ್ ದಾ ▁ ಮಾಲಾ ▁ಉದ್ ಧಾ ▁ಮ ದಾ ಕಾ ▁ಬ ಂಡ್ ರೋ ▁ಅವ ಗಾ ▁ಡೆ ಲೆ ಟೆಡ್
y: CategoryList
0,1,0,0,0
Path: .;

Valid: LabelList (777 items)
x: TextList
▁x x b os ▁100 ▁ಡೇ ಸ್ ▁ಪಕ್ಕ ಾ,▁x x b os ▁ಸುಪರ್ ▁ಸಿ ರ,▁x x b os ▁ಹ್ಯಾಂಡ್ ಸ್ ▁ಉಪ,▁x x b os ▁ಸಾರಿ ▁ನೀವು ▁ವಿಡಿಯೋ ▁ನ ▁ರೋಸ್ ತ ▁ಮ ದಿ ▁ಅ ದ್ ರೇ ▁ಮ ದ್ವಾ ಗಾ ▁ಪು ಬ್ ಗ ▁ಅ ತಾ ವಾ ▁ಫ್ರೀ ▁ಫೈರ್ ▁ಗೇಮ್ಸ್ ▁ಹ ಕೋ ಂಡೋ ▁ ಮದ್ ಬೇ ದೀ ▁ಆ ▁ಗೇಮ್ಸ್ ▁ಕೂಡಾ ▁ಚೀನ ್,▁x x b os ▁ಕ್ರಿ ಶಾ ನಾ ▁ಶಾಪ ಾ ▁ಟಾ ಟ್ಟೆ ಲಿ ಬೇಕು
y: CategoryList
0,1,1,0,0
Path: .;

Test: LabelList (778 i

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='f_beta', name='final')])

epoch,train_loss,valid_loss,accuracy,f_beta,time
0,0.782979,0.843966,0.705277,0.670101,00:20
1,0.704050,0.832813,0.711712,0.682713,00:21
2,0.640304,0.869732,0.700129,0.673620,00:19
3,0.481910,0.876074,0.706564,0.687821,00:21
4,0.400124,0.944344,0.722008,0.701671,00:19
5,0.291729,1.008146,0.732304,0.706932,00:20
6,0.207798,1.082946,0.714286,0.695666,00:19
7,0.148367,1.125818,0.718147,0.696955,00:20
8,0.135963,1.096886,0.719434,0.698204,00:18
9,0.125031,1.107432,0.722008,0.699895,00:20


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 0 with f_beta value: 0.6701013445854187.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 1 with f_beta value: 0.6827126145362854.


...

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 4 with f_beta value: 0.7016714811325073.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 5 with f_beta value: 0.706932008266449.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ar

In [25]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (6217 items)
x: TextList
▁x x b os ▁ ಟಿಕ ▁ಟೋ ಕ ▁ಅಲ್ಲ ೀ ▁ಜಗ ಲಾ ▁ಮಡ ಇ ದ್ರ ಲ್ಲಾ ▁ಆದ ್ರಾ ▁ಬಗ್ಗ ೇ ನು ಂ ▁ಓ ಂದು ▁ವಿಡಿಯೋ ▁ಮ ದಿ ▁ಅನ್ನಾ ▁ಸುಪರ್ ▁ಅ ಗೀ ▁ಉ ಗೀ ದೀ ದ್ದ ೀ ರಾ,▁x x b os ▁ಅನ್ಯ ೋ ನೆ ▁ಫ್ರ ್ಮ ▁ಕೇರಳ ಾ ▁ಹೀ ರಾ,▁x x b os ▁ಮೂ ವಿ ▁ರೇ ರೆ ಲೀಸ್ ▁ಮ ದಿ ▁ಪ್ ಲ್ಸ್,▁x x b os ▁ಅ ಮಾ ಝೋ ನ್ ▁ಪ್ ಹೈ ಮ ▁ಅಲ್ಲ ೀ ▁ಬಿಟ್ಟ ಿದ ಿ ರಾ ▁x x re p ▁4 ▁ . ▁ಯ ೆಲ್ಲಾ ▁ಮ ನೇ ಲೆ ▁ನೋಡ ್ ತಾ ರೇ ▁x x re p ▁4 ▁ .,▁x x b os ▁ಗುರು ▁ಸೂ ರೇ ▁ನ್ಯೂಸ್ ▁ನ ನು ▁ ಟಿಕ ▁ಟೋ ಕ ▁ದಾನ ▁ಲೋಡ್ ▁ಮ ದೇ ದಾ ▁ಯ ವಾಗ ಾ ▁ನಾಮ ಾ ▁ ನ್ ದ ುದು ▁ತಿಂದು ▁ನಾಮ ಾ ▁ದ ಶ್ ದಾ ▁ ಮಾಲಾ ▁ಉದ್ ಧಾ ▁ಮ ದಾ ಕಾ ▁ಬ ಂಡ್ ರೋ ▁ಅವ ಗಾ ▁ಡೆ ಲೆ ಟೆಡ್
y: CategoryList
0,1,0,0,0
Path: .;

Valid: LabelList (777 items)
x: TextList
▁x x b os ▁100 ▁ಡೇ ಸ್ ▁ಪಕ್ಕ ಾ,▁x x b os ▁ಸುಪರ್ ▁ಸಿ ರ,▁x x b os ▁ಹ್ಯಾಂಡ್ ಸ್ ▁ಉಪ,▁x x b os ▁ಸಾರಿ ▁ನೀವು ▁ವಿಡಿಯೋ ▁ನ ▁ರೋಸ್ ತ ▁ಮ ದಿ ▁ಅ ದ್ ರೇ ▁ಮ ದ್ವಾ ಗಾ ▁ಪು ಬ್ ಗ ▁ಅ ತಾ ವಾ ▁ಫ್ರೀ ▁ಫೈರ್ ▁ಗೇಮ್ಸ್ ▁ಹ ಕೋ ಂಡೋ ▁ ಮದ್ ಬೇ ದೀ ▁ಆ ▁ಗೇಮ್ಸ್ ▁ಕೂಡಾ ▁ಚೀನ ್,▁x x b os ▁ಕ್ರಿ ಶಾ ನಾ ▁ಶಾಪ ಾ ▁ಟಾ ಟ್ಟೆ ಲಿ ಬೇಕು
y: CategoryList
0,1,1,0,0
Path: .;

Test: LabelList (778 i

In [26]:
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat

df_test = data.val_df.copy()
from sklearn.metrics import f1_score
df_dict = {'query': list(df_test['input']), 'actual_label': list(df_test['label']), 'predicted_label': [0]*df_test.shape[0]}  
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Valid, ordered=True)
df_result['predicted_label'] = list(get_predicted(np.array(preds[0])))
df_result.head()

,query,actual_label,predicted_label
0,100 ಡೇಸ್ ಪಕ್ಕಾ,0,1
1,ಸುಪರ್ ಸಿರ,1,1
2,ಹ್ಯಾಂಡ್ಸ್ ಉಪ,1,1
3,ಸಾರಿ ನೀವು ವಿಡಿಯೋ ನ ರೋಸ್ತ ಮದಿ ಅದ್ರೇ ಮದ್ವಾಗಾ ಪುಬ...,0,3
4,ಕ್ರಿಶಾನಾ ಶಾಪಾ ಟಾಟ್ಟೆಲಿಬೇಕು,0,0


In [27]:
df_result.head()

,query,actual_label,predicted_label
0,100 ಡೇಸ್ ಪಕ್ಕಾ,0,1
1,ಸುಪರ್ ಸಿರ,1,1
2,ಹ್ಯಾಂಡ್ಸ್ ಉಪ,1,1
3,ಸಾರಿ ನೀವು ವಿಡಿಯೋ ನ ರೋಸ್ತ ಮದಿ ಅದ್ರೇ ಮದ್ವಾಗಾ ಪುಬ...,0,3
4,ಕ್ರಿಶಾನಾ ಶಾಪಾ ಟಾಟ್ಟೆಲಿಬೇಕು,0,0


In [28]:
f1_score(df_result['actual_label'], df_result['predicted_label'], average = 'weighted')

0.7069320018093924

In [29]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

NameError: ignored

In [30]:
logits = {}
preds = learn.get_preds(ds_type=DatasetType.Valid, ordered=True)
logits['val'] = preds[0]
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
logits['test'] = preds[0]

In [31]:
logits

{'test': tensor([[9.9828e-01, 5.3023e-04, 7.6019e-04, 1.1040e-04, 1.0478e-04, 2.1709e-04],
         [6.8540e-01, 3.6731e-04, 2.1867e-01, 1.6670e-02, 4.4085e-02, 3.4811e-02],
         [9.9879e-01, 1.6429e-04, 7.6156e-05, 5.3310e-05, 8.7152e-04, 4.2262e-05],
         ...,
         [4.8709e-02, 9.4163e-01, 7.1554e-04, 1.4756e-03, 3.8375e-03, 3.6322e-03],
         [9.6575e-01, 3.1003e-03, 1.9744e-02, 3.3069e-03, 1.2406e-03, 6.8557e-03],
         [9.9489e-01, 1.1795e-04, 4.4710e-03, 8.4560e-05, 1.3530e-04, 3.0518e-04]]),
 'val': tensor([[3.3567e-01, 6.6109e-01, 4.9146e-04, 3.1171e-04, 1.1206e-04, 2.3194e-03],
         [5.6065e-02, 9.3795e-01, 6.6987e-04, 1.1268e-03, 1.5285e-03, 2.6578e-03],
         [1.4794e-01, 8.4308e-01, 4.0108e-03, 2.0148e-04, 6.8246e-04, 4.0827e-03],
         ...,
         [2.1828e-02, 9.7469e-01, 4.7421e-04, 5.5840e-04, 2.9439e-04, 2.1506e-03],
         [9.9954e-01, 1.0447e-04, 1.1870e-04, 7.7677e-05, 6.0102e-05, 1.0039e-04],
         [6.7510e-01, 5.7965e-05, 1.4682e-

In [32]:
with open('kan_ulmfit_preds.pkl', 'wb') as f:
    pickle.dump(logits, f)

In [33]:
with open('kan_ulmfit_preds.pkl', 'rb') as f:
    dpred = pickle.load(f)

In [34]:
dpred

{'test': tensor([[9.9828e-01, 5.3023e-04, 7.6019e-04, 1.1040e-04, 1.0478e-04, 2.1709e-04],
         [6.8540e-01, 3.6731e-04, 2.1867e-01, 1.6670e-02, 4.4085e-02, 3.4811e-02],
         [9.9879e-01, 1.6429e-04, 7.6156e-05, 5.3310e-05, 8.7152e-04, 4.2262e-05],
         ...,
         [4.8709e-02, 9.4163e-01, 7.1554e-04, 1.4756e-03, 3.8375e-03, 3.6322e-03],
         [9.6575e-01, 3.1003e-03, 1.9744e-02, 3.3069e-03, 1.2406e-03, 6.8557e-03],
         [9.9489e-01, 1.1795e-04, 4.4710e-03, 8.4560e-05, 1.3530e-04, 3.0518e-04]]),
 'val': tensor([[3.3567e-01, 6.6109e-01, 4.9146e-04, 3.1171e-04, 1.1206e-04, 2.3194e-03],
         [5.6065e-02, 9.3795e-01, 6.6987e-04, 1.1268e-03, 1.5285e-03, 2.6578e-03],
         [1.4794e-01, 8.4308e-01, 4.0108e-03, 2.0148e-04, 6.8246e-04, 4.0827e-03],
         ...,
         [2.1828e-02, 9.7469e-01, 4.7421e-04, 5.5840e-04, 2.9439e-04, 2.1506e-03],
         [9.9954e-01, 1.0447e-04, 1.1870e-04, 7.7677e-05, 6.0102e-05, 1.0039e-04],
         [6.7510e-01, 5.7965e-05, 1.4682e-